In [1]:
import torch
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [5]:
# 设置 GPU 设备
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# 设定随机种子
def setup_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

setup_seed(888888)

print(device)

# 基础参数
h = 100    # 画图网格密度
N = 1000    # 内点配置点数
N1 = 0.01  # 边界点配置点数
N2 = 1000  # PDE数据点

# 定义 MLP 模型
class MLP(torch.nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.net = torch.nn.Sequential(
            torch.nn.Linear(2, 32),
            torch.nn.Tanh(),
            torch.nn.Linear(32, 32),
            torch.nn.Tanh(),
            torch.nn.Linear(32, 32),
            torch.nn.Tanh(),
            torch.nn.Linear(32, 32),
            torch.nn.Tanh(),
            torch.nn.Linear(32, 1)
        )

    def forward(self, x):
        return self.net(x)

# 定义模型并加载权重
u = MLP().to(device)
u.load_state_dict(torch.load('weights_4D.pth', map_location=device))
u.eval()  # 设置模型为评估模式

cpu


MLP(
  (net): Sequential(
    (0): Linear(in_features=2, out_features=32, bias=True)
    (1): Tanh()
    (2): Linear(in_features=32, out_features=32, bias=True)
    (3): Tanh()
    (4): Linear(in_features=32, out_features=32, bias=True)
    (5): Tanh()
    (6): Linear(in_features=32, out_features=32, bias=True)
    (7): Tanh()
    (8): Linear(in_features=32, out_features=1, bias=True)
  )
)

In [6]:
# 生成 [-1,1] x [-1,1] 计算网格
x = torch.linspace(0, 1, h)
y = torch.linspace(0, 1, h)
xx, yy = torch.meshgrid(x, y, indexing="ij")

# 计算解析解
u_real = torch.relu(torch.sin(torch.pi * xx) * torch.sin(torch.pi * yy)).to(device)

# 计算 MLP 预测解
grid_points = torch.stack([xx.flatten(), yy.flatten()], dim=1).to(device)  # (h*h, 2)
u_pred = u(grid_points)  # 预测值 (h*h, 1)
u_pred = torch.relu(u_pred)

# 重新将预测解恢复为二维网格 (h, h)
u_pred_grid = u_pred.view(h, h)

# 输出形状检查
print(f"解析解形状: {u_real.shape}")  # 应该是 (h, h)
print(f"模型预测形状: {u_pred_grid.shape}")  # 应该是 (h, h)


解析解形状: torch.Size([100, 100])
模型预测形状: torch.Size([100, 100])


In [7]:
# 计算误差
batch_size = 1000  # 每批计算1000个点
num_batches = u_pred_grid.shape[0] * u_pred_grid.shape[1] // batch_size  # 分批次数

# 初始化误差指标
max_error = 0.0
mae = 0.0  # MAE
mse = 0.0  # MSE
l2_norm_u_real = torch.norm(u_real) ** 2  # 解析解的 L2 范数平方，用于相对误差计算

# 逐批计算误差
for i in range(num_batches):
    start = i * batch_size
    end = start + batch_size
    batch_pred = u_pred[start:end].view(-1)  # 拉平为一维
    batch_real = u_real.flatten()[start:end].view(-1)  # 拉平为一维

    batch_error = torch.abs(batch_pred - batch_real)
    
    # 更新最大误差
    max_batch_error = torch.max(batch_error)
    max_error = max(max_error, max_batch_error)

    # 更新 MAE 和 MSE
    mae += torch.sum(batch_error)
    mse += torch.sum(batch_error ** 2)

# 处理剩余部分（如果不能被batch_size整除）
if u_pred.shape[0] % batch_size != 0:
    start = num_batches * batch_size
    batch_pred = u_pred[start:].view(-1)
    batch_real = u_real.flatten()[start:].view(-1)
    batch_error = torch.abs(batch_pred - batch_real)

    # 更新最大误差
    max_batch_error = torch.max(batch_error)
    max_error = max(max_error, max_batch_error)

    # 更新 MAE 和 MSE
    mae += torch.sum(batch_error)
    mse += torch.sum(batch_error ** 2)

# 计算总点数
total_points = u_pred_grid.shape[0] * u_pred_grid.shape[1]

# 计算 MAE 和 MSE
mae /= total_points
mse /= total_points

# 计算相对 L2 范数误差
l2_norm_error = torch.norm(u_pred_grid - u_real) ** 2 / l2_norm_u_real

# 输出结果
print(f"最大误差 Max Error: {max_error.item()}")
print(f"MAE (Mean Absolute Error): {mae.item()}")
print(f"MSE (Mean Squared Error): {mse.item()}")
print(f"Relative L2 Norm Error: {l2_norm_error.item()}")

最大误差 Max Error: 0.0008835792541503906
MAE (Mean Absolute Error): 0.000296463054837659
MSE (Mean Squared Error): 1.464542265239288e-07
Relative L2 Norm Error: 5.977117893962713e-07


In [ ]:
import torch
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# 设置 GPU 设备
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# 基础参数
h = 100  # 网格密度

# 生成 [-1,1] x [-1,1] 计算网格
x = torch.linspace(-1, 1, h)
y = torch.linspace(-1, 1, h)
xx, yy = torch.meshgrid(x, y, indexing="ij")

# 计算解析解
u_real = torch.relu(torch.sin(torch.pi * xx) * torch.sin(torch.pi * yy)).to(device)

# 计算 MLP 预测解
grid_points = torch.stack([xx.flatten(), yy.flatten()], dim=1).to(device)  # (h*h, 2)
u_pred = u(grid_points)  # 预测值 (h*h, 1)

# 将预测解和解析解恢复成网格形式
u_real_grid = u_real.cpu().detach().numpy().reshape(h, h)
u_pred_grid = u_pred.cpu().detach().numpy().reshape(h, h)

# 绘制解析解和模型解的 3D 图
fig = plt.figure(figsize=(18, 8))

# 绘制解析解
ax1 = fig.add_subplot(121, projection='3d')
ax1.plot_surface(xx.cpu().numpy(), yy.cpu().numpy(), u_real_grid, cmap='viridis')
ax1.set_title('Analytical Solution')
ax1.set_xlabel('X')
ax1.set_ylabel('Y')
ax1.set_zlabel('u_real')

# 绘制模型解
ax2 = fig.add_subplot(122, projection='3d')
ax2.plot_surface(xx.cpu().numpy(), yy.cpu().numpy(), u_pred_grid, cmap='viridis')
ax2.set_title('Model Prediction')
ax2.set_xlabel('X')
ax2.set_ylabel('Y')
ax2.set_zlabel('u_pred')

# 显示图形
plt.show()
